In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import confusion_matrix

#VIF để phát hiện đa cộng tuyến trong mô hình hồi quy.
from statsmodels.stats.outliers_influence import variance_inflation_factor

**GIẢI THÍCH DỮ LIỆU:**

**1.order_id:** Mã định danh duy nhất cho mỗi đơn hàng.

**2.product_id:** Mã định danh duy nhất cho mỗi sản phẩm.

**3.add_to_cart_order:** Thứ tự mà sản phẩm được thêm vào giỏ hàng trong một đơn hàng cụ thể.

**4,.reordered:** Biến nhị phân (0 hoặc 1) cho biết sản phẩm này có được đặt hàng lại hay không (1 nếu đã được đặt hàng trước đó, 0 nếu đây là lần đầu tiên).

**5.user_id:** Mã định danh duy nhất cho mỗi khách hàng.

**6.eval_set:** Loại tập dữ liệu mà đơn hàng thuộc về (ví dụ: 'prior', 'train', 'test'). Dùng để xác định liệu đơn hàng đó nằm trong tập dữ liệu trước đây, tập huấn luyện hay tập kiểm tra.

**7.order_number:** Thứ tự của đơn hàng của khách hàng, tức là đơn hàng thứ bao nhiêu của khách hàng đó.

**8.order_dow:**  Ngày trong tuần mà đơn hàng được đặt (0 cho Chủ Nhật, 1 cho Thứ Hai, v.v.).

**9.order_hour_of_day:** Giờ trong ngày mà đơn hàng được đặt (từ 0 đến 23).

**10.days_since_prior_order:** Số ngày kể từ đơn hàng trước đó của khách hàng. Dùng để phân tích tần suất mua sắm của khách hàng.

**11.product_name:** Tên của sản phẩm.

**12.aisle_id:** Mã định danh duy nhất cho mỗi ngăn kệ trong cửa hàng.

**13.aisle:** Tên của ngăn kệ trong cửa hàng (ví dụ: 'beverages', 'snacks').

**14.department_id:** Mã định danh duy nhất cho mỗi bộ phận trong cửa hàng.

**15.department:** Tên của bộ phận trong cửa hàng (ví dụ: 'produce', 'bakery').

In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/departments.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/order_products__train.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/order_products__prior.csv')
orders = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/Datasets/instacart-market-basket-analysis/products.csv')

In [ ]:
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [ ]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [ ]:
order_products_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [ ]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [ ]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


***1.Tạo dataframe cho tập dữ liệu Train:***

In [ ]:
#Lọc ra tất cả các nhãn 'train' của tập orders:
train_orders = orders[orders['eval_set'] == 'train']

In [ ]:
#Hợp nhất 2 cột order_products_train và train_orders thông qua order_id
train_data = order_products_train.merge(train_orders , on = 'order_id')

In [ ]:
#Hợp nhất 2 cột products, aisles:
products_ailse = products.merge(aisles, on = 'aisle_id')

In [ ]:
#Hợp nhất cả 3 cột products, aisles và departments:
products_ailse_department = products_ailse.merge(departments, on = 'department_id')

In [ ]:
#Hợp nhất tất cả các cột thành tập train_data
train_data = train_data.merge(products_ailse_department , on = 'product_id')

In [ ]:
train_data

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,47901,train,14,4,6,16.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,2993,train,15,0,7,7.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,41425,train,4,3,8,14.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,187205,train,5,4,14,30.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,177077,train,4,4,16,30.0,Sprinkles Decors,97,13,baking supplies decor,pantry
1384613,3420084,47935,20,0,9808,train,9,6,21,30.0,Classic Original Lip Balm SPF 12,73,11,facial care,personal care
1384614,3420084,9491,21,0,9808,train,9,6,21,30.0,Goats Milk & Chai Soap,25,11,soap,personal care
1384615,3420088,16380,12,0,72444,train,14,1,8,5.0,Stevia Sweetener,97,13,baking supplies decor,pantry


In [ ]:
#Bỏ đi một số cột không cần thiết
train_data = train_data.drop(columns=['eval_set', 'product_name', 'aisle', 'department'])

In [ ]:
train_data.isnull().sum()

order_id                  0
product_id                0
add_to_cart_order         0
reordered                 0
user_id                   0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
aisle_id                  0
department_id             0
dtype: int64

In [ ]:
train_data.to_csv('Instacart-market-basket-train.csv', index=False)

***2.Tạo dataframe cho tập dữ liệu Test:***

In [ ]:
#Lọc ra tất cả các nhãn 'test' của tập orders:
test_orders = orders[orders['eval_set'] == 'test']

In [ ]:
test_data = test_orders.drop(columns=['eval_set'])

In [ ]:
test_data.to_csv('Instacart-market-basket-test.csv', index=False)